<a href="https://colab.research.google.com/github/rlagosb/taller_eiv/blob/main/0_Ejemplos_Anonimizaci%C3%B3n_Programacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Examinemos dos ejemplos de datos para la programación y apliquemos las orientaciones de la **[Norma técnica de anonimización para la publicación bases de datos como datos abiertos](https://repositoriodeis.minsal.cl/ContenidoSitioWeb2020/EstandaresNormativa/Norma%20t%C3%A9cnica%20241%20de%20anonimizaci%C3%B3n%20datos%20abiertos.pdf)**.  del Departamento de Estadísticas e Información de Salud (2025).


# Ejemplo 1: Fichas de Programación

En este script extraemos la programación de un conjunto de profesionales de un servicio clínico y guardamos los datos originales, seudoanonimizados y anonimizados



> ***Anonimización**: Procedimiento en virtud del cual un dato personal no puede
vincularse o asociarse a una persona determinada, ni permitir su  identificación, por haberse destruido o eliminado el nexo con la información que vincula, asocia o identifica a esa persona. Un dato anonimizado deja de ser un dato personal.*

> ***Seudoanonimización**: es el tratamiento de datos personales de manera tal que ya no puede atribuirse a un titular sin utilizar información adicional, siempre que dicha información adicional figure por separado y esté sujeta a medidas técnicas y organizativas destinadas a garantizar que los datos personales no se atribuyan a una persona física identificada o identificable. Este proceso es reversible, en cuanto que,al juntar información adicional con los datos personales seudoanonimizados, se podrá volver a atribuir ese dato a una persona identificada o identificable. Este proceso se denomina reidentificación.*

In [1]:
#@title Descargar Planilla

import pandas as pd
import openpyxl as xl
import requests
import os

# Localización de los datos
path_fichas="https://raw.githubusercontent.com/rlagosb/taller_eiv/main/data/programacion/Fichas/"
planilla_fichas = 'Fichas Unidad Endoscopia.xlsx'

# Descargar el archivo
response = requests.get(path_fichas + planilla_fichas)
with open(planilla_fichas, 'wb') as f:
    f.write(response.content)

# Cargar planilla
wb = xl.load_workbook(filename = planilla_fichas, data_only=True)

print(f'Planilla {planilla_fichas} descargada ✅')

/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Planilla Fichas Unidad Endoscopia.xlsx descargada ✅


In [2]:
#@title Obtener fichas

# Cargar datos de las fichas en las hojas de la planilla
fichas = {}
print('Cargando fichas:')

for ws_name in [name for name in wb.sheetnames if name not in ['Personal', 'Prestaciones', 'Ficha Programación']]:

  #Cargar hoja
  ws=wb[ws_name]

  #obtener datos profesional y eliminar cabecera
  profesional={'rut':ws['C4'].value,'nombre':ws['C3'].value,'profesion':ws['C5'].value,'obs_ficha':ws['F5'].value}
  ws.delete_rows(1,9)
  ws.delete_cols(1,1)

  #obtener actividades y horarios
  prog = (pd.DataFrame(ws.values).rename(columns={0:"ini",1:"fin"}).
          melt(id_vars=['ini','fin'], var_name='dia', value_name='act').
          # eliminar bloques sin actividades programadas
          dropna(subset=['act']))
  prog['dia']=prog['dia']-1

  #agregar datos profesional
  for col in ['rut','nombre','profesion','obs_ficha']:
    prog[col]=profesional[col]

  #agrupar bloques de actividades
  prog['id_prog']=((prog.rut!=prog.rut.shift()) |             # agrupar por cambio de rut
                    (prog.dia!=prog.dia.shift()) |            # por cambio de dia
                    (prog.act!=prog.act.shift())              # por cambio de actividad
                    ).cumsum()                                # asignar correlativo
  fichas[ws_name]=prog.groupby(['id_prog','rut','nombre','profesion','dia','act','obs_ficha'],as_index=False, dropna=False).aggregate({'ini':'min','fin':'max'})

  print(ws.title, '✔')



Cargando fichas:
OF ✔
JH ✔
VR ✔
EG ✔
VJ ✔
JG ✔
MG ✔
LR ✔
DR ✔


In [3]:
#@title Tabular datos

#Consolidar  fichas en una tabla
programacion=pd.concat(fichas, ignore_index=True)

# Calcular duración actividades
programacion['hrs_prog']=(pd.to_datetime(programacion['fin'].astype(str), format='%H:%M:%S') -
                     pd.to_datetime(programacion['ini'].astype(str), format='%H:%M:%S')).dt.total_seconds() / 3600

programacion

,id_prog,rut,nombre,profesion,dia,act,obs_ficha,ini,fin,hrs_prog
0,1,123456,Orlando Flores,Médico(a) Cirujano(a),1,[GA] Consulta nueva,Comisión de Servicio,08:00:00,10:00:00,2.0
1,2,123456,Orlando Flores,Médico(a) Cirujano(a),1,[GA] Otras Actividades Clínicas,Comisión de Servicio,10:00:00,14:00:00,4.0
2,3,123456,Orlando Flores,Médico(a) Cirujano(a),1,[GA] Colonoscopía,Comisión de Servicio,14:00:00,15:00:00,1.0
3,4,123456,Orlando Flores,Médico(a) Cirujano(a),1,[GA] Endoscopía digestiva,Comisión de Servicio,15:00:00,16:00:00,1.0
4,5,123456,Orlando Flores,Médico(a) Cirujano(a),2,[GA] Otras Actividades Clínicas,Comisión de Servicio,08:00:00,09:00:00,1.0
...,...,...,...,...,...,...,...,...,...,...
128,12,369874,Daniela Ruiz,Médico(a) Cirujano(a),5,[GA] Comité Clínico,Encargada Unidad,08:30:00,10:30:00,2.0
129,13,369874,Daniela Ruiz,Médico(a) Cirujano(a),5,[GA] Otras Actividades Clínicas,Encargada Unidad,10:30:00,13:00:00,2.5
130,14,369874,Daniela Ruiz,Médico(a) Cirujano(a),5,Colación,Encargada Unidad,13:00:00,13:30:00,0.5
131,15,369874,Daniela Ruiz,Médico(a) Cirujano(a),5,[GA] Otras Actividades Clínicas,Encargada Unidad,13:30:00,14:00:00,0.5


🔴 ¿Cuáles campos son identificadores explícitos? ¿Cuasi-identificadores?

> • ***Identificadores explícitos**. Son datos que permiten identificar de forma inequívoca a una persona como el nombre, número de identificación nacional, [...] número de teléfono móvil.*

> • ***Cuasi-identificadores**. Son datos que no permiten una identificación directa del individuo, pero que en conjunto con otros datos pueden llegar a señalar a la persona como sexo, género, fecha de nacimiento, edad, ocupación, estado civil, nacionalidad, lugar de atención, comuna, región, previsión, fecha de egreso, entre otros.*

> • ***Atributos sensibles**: Son datos que revelan características físicas o morales y que pueden comprometer la privacidad de los individuos como los diagnósticos, procedimientos clínicos, estado de vacunación, entre otros.*

In [4]:
#@title Des-identificación
# Cremos un identificador alternativo

profesionales = programacion[['rut']].drop_duplicates(ignore_index=True).reset_index().rename(columns={'index':'idProfesional'})
profesionales['idProfesional'] +=1
print('Identificadores de Profesionales\n', profesionales)

# Agregamos el pseudoidentificador y eliminamos los datos personales

programacion_desidentificada = programacion.merge(profesionales, on='rut', how='left').drop(columns=['rut','nombre','profesion'])
programacion_desidentificada

Identificadores de Profesionales
    idProfesional     rut
0              1  123456
1              2  678910
2              3  321654
3              4  987321
4              5  951843
5              6  357681
6              7  852963
7              8  147258
8              9  369874


,id_prog,dia,act,obs_ficha,ini,fin,hrs_prog,idProfesional
0,1,1,[GA] Consulta nueva,Comisión de Servicio,08:00:00,10:00:00,2.0,1
1,2,1,[GA] Otras Actividades Clínicas,Comisión de Servicio,10:00:00,14:00:00,4.0,1
2,3,1,[GA] Colonoscopía,Comisión de Servicio,14:00:00,15:00:00,1.0,1
3,4,1,[GA] Endoscopía digestiva,Comisión de Servicio,15:00:00,16:00:00,1.0,1
4,5,2,[GA] Otras Actividades Clínicas,Comisión de Servicio,08:00:00,09:00:00,1.0,1
...,...,...,...,...,...,...,...,...
128,12,5,[GA] Comité Clínico,Encargada Unidad,08:30:00,10:30:00,2.0,9
129,13,5,[GA] Otras Actividades Clínicas,Encargada Unidad,10:30:00,13:00:00,2.5,9
130,14,5,Colación,Encargada Unidad,13:00:00,13:30:00,0.5,9
131,15,5,[GA] Otras Actividades Clínicas,Encargada Unidad,13:30:00,14:00:00,0.5,9


In [5]:
# @title Anonimización
# Eliminamos los campos identificadores y pseudoidentificadores, dejando sólos los campos indispensables

programacion_anonimizada = programacion.groupby(['profesion','act'], dropna=False).hrs_prog.sum().reset_index()
programacion_anonimizada


,profesion,act,hrs_prog
0,Médico(a) Cirujano(a),Colación,5.0
1,Médico(a) Cirujano(a),[GA] Colonoscopía,29.5
2,Médico(a) Cirujano(a),[GA] Comité Clínico,15.5
3,Médico(a) Cirujano(a),[GA] Consulta control,1.0
4,Médico(a) Cirujano(a),[GA] Consulta nueva,43.5
5,Médico(a) Cirujano(a),[GA] Consulta nueva+docencia,7.0
6,Médico(a) Cirujano(a),[GA] Endoscopía digestiva,22.0
7,Médico(a) Cirujano(a),[GA] Otras Actividades Clínicas,63.0
8,Médico(a) Cirujano(a),[GA] Procedimientos especialidad,16.5
9,Médico(a) Cirujano(a),[GA] Reunión Clínica,6.0


> 🟡 *Muchas veces se confunde la desidentificación con la anonimización,
pero la desidentificación es sólo una técnica de un conjunto de procedimientos a aplicar a la base de datos y, aplicada por si sola, genera un conjunto de datos que puede ser identificable al combinarlos con otros datos de acceso público. Este proceso se denomina *reidentificación* y es lo que se busca
evitar con la anonimización.*

> 🔴 *Si la base de datos contiene variables registradas en texto libre pueden existir identificadores explícitos contenidos en estos campos que no hayan sido reconocidos. Esta posibilidad aumenta cuando el volumen de información es masivo, por lo que estas variables también deben eliminarse previo al proceso de publicación como datos abiertos, mientras no se pueda asegurar la anonimización del campo.*

In [7]:
#@title Exportar

# Carpeta de destino
datos_crudos = '/content/datos_crudos/'
if not os.path.exists(datos_crudos): os.makedirs(datos_crudos)

# Crear un ExcelWriter para exportar a múltiples hojas
with pd.ExcelWriter(datos_crudos + "Programacion.xlsx") as writer:
    programacion.to_excel(writer, sheet_name="PM", index=False)
    programacion_desidentificada.to_excel(writer, sheet_name="PM_Pseudoanonimizada", index=False)
    programacion_anonimizada.to_excel(writer, sheet_name="PM_Anonimizada", index=False)

print(f"Dataframes exportados a {datos_crudos}Programacion.xlsx ✅")

Dataframes exportados a /content/datos_crudos/Programacion.xlsx ✅


# Ejemplo 2: Interconsultas No GES

In [8]:
# @title Cargar planilla

# concatenamos para definir la dirección excel_url, la cual almacena la dirección del archivo a analizar.
datos_originales = 'https://raw.githubusercontent.com/rlagosb/taller_eiv/main/data/'
excel_url = datos_originales + 'SS_MSOr_Respuesta%20Solicitud%20Folio%20AO012T0001655%20(CNE%202021).xlsx'

# importamos desde esa url las hojas del excel LE ABIERTA y CNE EGRESOS
sic_df={}
for hoja in ['LE ABIERTA ','CNE EGRESOS']:
  sic_df[hoja]=pd.read_excel(excel_url, sheet_name=hoja)
  print(f'Hoja {hoja}: {sic_df[hoja].shape[0]} registros')


# Crear tabla única
sic = pd.concat(sic_df.values())

print(sic.info())
sic

Hoja LE ABIERTA : 2499 registros
Hoja CNE EGRESOS: 149479 registros
<class 'pandas.core.frame.DataFrame'>
Index: 151978 entries, 0 to 149478
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   FECHA_NAC   151978 non-null  object        
 1   SEXO        151978 non-null  int64         
 2   TIPO_PREST  151978 non-null  int64         
 3   PRESTA_MIN  151978 non-null  object        
 4   PRESTA_EST  151978 non-null  object        
 5   F_ENTRADA   151978 non-null  datetime64[ns]
 6   ESTAB_ORIG  151978 non-null  int64         
 7   ESTAB_DEST  151978 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 10.4+ MB
None


,FECHA_NAC,SEXO,TIPO_PREST,PRESTA_MIN,PRESTA_EST,F_ENTRADA,ESTAB_ORIG,ESTAB_DEST
0,2010-11-22 00:00:00,2,1,07-064,NUTRIOLOGO PEDIATRICO,2021-01-04,114101,114101
1,1987-12-17 00:00:00,2,1,07-056,CIRUGIA ABDOMINAL,2021-01-04,114312,114101
2,1988-09-07 00:00:00,2,1,07-058,GINECOLOGIA ADULTO,2021-01-04,114101,114101
3,1964-07-14 00:00:00,2,1,09-008,ORTODONCIA,2021-01-04,114101,114101
4,2017-05-18 00:00:00,1,1,09-008,Ortodoncia,2021-01-04,114304,114105
...,...,...,...,...,...,...,...,...
149474,27260,2,1,07-030,MEDICINA INTERNA,2021-11-25,114322,114101
149475,43382,1,1,07-049,Psiquiatria,2021-06-17,114105,114105
149476,35875,1,1,07-046,Otorrinolaringología,2021-05-12,114303,114331
149477,26707,1,1,07-046,Otorrinolaringología,2021-12-28,114302,114331


🔴 Esta base no tiene identificadores explícitos pero tiene cuasi-identificadores. No debiera existir una combinación única de cuasi-identificadores entre todas las filas de la base de datos 🔍

In [9]:
sic['EDAD'] = (sic['F_ENTRADA'] - pd.to_datetime(sic['FECHA_NAC'])).dt.days // 365
sic[['EDAD','SEXO']].value_counts(dropna=False)

EDAD  SEXO
51    2       84427
      1       59972
52    2        3045
      1        2045
8     2          69
              ...  
92    1           1
93    2           1
94    2           1
96    2           1
99    2           1
Name: count, Length: 187, dtype: int64

🟢 **Generalización**: El objetivo de este procedimiento es obtener una k-anonimidad mayor a 1, donde el valor K corresponde a cuantas veces es lo mínimo que está repetido una tupla de cuasi-identificadores en el conjunto de datos. Para lograr esto se limita la precisión de los datos a través del establecimiento de una jerarquía en la que ciertos atributos del mismo grupo comparten valores.

In [10]:
# Para obtener una K-anonimidad mayor a 2 se aplicó una generalización de la variable edad por rangos etarios

sic['GRUPO_EDAD'] = pd.cut(sic['EDAD'], bins=range(0, 101, 10), right=False)
sic[['GRUPO_EDAD','SEXO']].value_counts(dropna=False)

GRUPO_EDAD  SEXO
[50, 60)    2       87632
            1       62121
[0, 10)     2         304
            1         237
[20, 30)    2         190
[60, 70)    2         187
[10, 20)    2         170
            1         151
[60, 70)    1         138
[30, 40)    2         133
[70, 80)    2         125
            1         112
[40, 50)    2         101
[20, 30)    1          88
[40, 50)    1          80
[30, 40)    1          78
[80, 90)    2          72
            1          28
[50, 60)    99         13
            3           9
[90, 100)   2           7
[60, 70)    3           1
[90, 100)   1           1
Name: count, dtype: int64

In [11]:
# Otra opción es enmascarar las variables para lograr una K-anonimidad mayor a 2
# Enmascaremos la variable Sexo con el valor 99 = desconocido en los casos con k-anonimidad=1

sic.loc[(sic.EDAD.between(60, 70, inclusive='left') & (sic.SEXO==3)), 'SEXO'] = 99
sic.loc[((sic.EDAD>=90) & (sic.SEXO==1)), 'SEXO'] = 99

sic[['GRUPO_EDAD','SEXO']].value_counts(dropna=False)

GRUPO_EDAD  SEXO
[50, 60)    2       87632
            1       62121
[0, 10)     2         304
            1         237
[20, 30)    2         190
[60, 70)    2         187
[10, 20)    2         170
            1         151
[60, 70)    1         138
[30, 40)    2         133
[70, 80)    2         125
            1         112
[40, 50)    2         101
[20, 30)    1          88
[40, 50)    1          80
[30, 40)    1          78
[80, 90)    2          72
            1          28
[50, 60)    99         13
            3           9
[90, 100)   2           7
[60, 70)    99          1
[90, 100)   99          1
Name: count, dtype: int64

In [12]:
# Eliminando la edad quedamos un conjunto de datos k-anonimizados (k=2)
del sic['EDAD']

> *Una vez obtenida una K-anonimidad igual o mayor a 2, se debe evaluar la L- diversidad que corresponde al número de valores distintos de los atributos sensibles que existen en una misma tupla única de cuasi-identificadores. Esto se debe a que si una tupla repetida K veces tiene el mismo atributo
sensible se puede identificar en la base de datos. Al igual que en la K-anonimidad, el valor de la L diversidad debe ser mayor a 1.*

## Exposición de datos sensibles
Consideremos que `PRESTA_MIN` es proxy del diagnóstico del paciente y, por lo tanto, es un atributo sensible (*aquellos datos personales que se refieren a las características físicas o morales de las personas o a hechos o circunstancias de su vida privada o intimidad*)



In [13]:
# Examinemos si los grupos de cuasi-identificadores tienen l-diversidad > 1

l_diversity_grupos = sic.groupby(['GRUPO_EDAD', 'SEXO'], observed=True)['PRESTA_MIN'].nunique().reset_index().rename(columns={'PRESTA_MIN': 'L_Diversidad'})

print("L-Diversity por grupo:\n", l_diversity_grupos)

if (l_diversity_grupos['L_Diversidad'] < 2).any():
    print("Grupos sin L-diversidad ≥ 2:\n", l_diversity_grupos[l_diversity_grupos['L_Diversidad'] < 2])

L-Diversity por grupo:
    GRUPO_EDAD  SEXO  L_Diversidad
0     [0, 10)     1            18
1     [0, 10)     2            13
2    [10, 20)     1            20
3    [10, 20)     2            21
4    [20, 30)     1            14
5    [20, 30)     2            26
6    [30, 40)     1            23
7    [30, 40)     2            26
8    [40, 50)     1            22
9    [40, 50)     2            20
10   [50, 60)     1            66
11   [50, 60)     2            65
12   [50, 60)     3             8
13   [50, 60)    99            11
14   [60, 70)     1            19
15   [60, 70)     2            26
16   [60, 70)    99             1
17   [70, 80)     1            18
18   [70, 80)     2            20
19   [80, 90)     1            12
20   [80, 90)     2            14
21  [90, 100)     2             4
22  [90, 100)    99             1
Grupos sin L-diversidad ≥ 2:
    GRUPO_EDAD  SEXO  L_Diversidad
16   [60, 70)    99             1
22  [90, 100)    99             1


## Resumen

*   Los cuasi-identificadores del dataframe `sic` son `GRUPO_EDAD` y `SEXO`.
*   El dato sensible identificado es `PRESTA_MIN`.
*   La L-diversidad (número de valores posibles del dato sensible) fue calculada para cada grupo formado por los cuasi-identificadores.
*   Sólo dos grupos no alcanzaron L-diversity = 2, casos con sexo desconocido. Sin embargo, los grupos con mismo rango etario y sexo conocido si tuvieron l-diversidad>1.
*   La base alcanza L-diversidad=4, ya que todos los grupos de cuasi-identificadores tienen al menos cuatro posibles valores del atributo sensible.

## Conclusión
✅ La anonimización utilizando generalización de `GRUPO_EDAD` y ofuscasión de `SEXO` protege contra la reidentificación y exposición de datos sensibles (`PRESTA_MIN`).